In [1]:
#REFS:
#Wildon, Vertices of Specht Modules and Blocks of the Symmetric Group
#Murphy, The Idempotents of the Symmetric Group and Nakayama's Conjecture
#Murphy, A new construction of Young's seminormal representation of the symmetric groups

In [2]:
SGA_Q3 = SymmetricGroupAlgebra(QQ,3)

In [3]:
SGA_Q3.dft()

[   1    1    1    1    1    1]
[   1  1/2   -1 -1/2 -1/2  1/2]
[   0  3/4    0  3/4 -3/4 -3/4]
[   0    1    0   -1    1   -1]
[   1 -1/2    1 -1/2 -1/2 -1/2]
[   1   -1   -1    1    1   -1]

In [4]:
SGA_F7 = SymmetricGroupAlgebra(GF(7),3)

In [5]:
SGA_F7.dft()

[1 1 1 1 1 1]
[1 4 6 3 3 4]
[0 6 0 6 1 1]
[0 1 0 6 1 6]
[1 3 1 3 3 3]
[1 6 6 1 1 6]

In [6]:
SGA_F7.specht_module(Partition([1,1,1]))

Specht module of [(0, 0), (1, 0), (2, 0)] over Finite Field of size 7

In [7]:
SGA_F3 = SymmetricGroupAlgebra(GF(3),3)

In [8]:
#one cannot perform the DFT when p | n!
try:
    SGA_F3.dft()
except ZeroDivisionError:
    print("Modular case not handled!")

Modular case not handled!


In [9]:
#construct Pierce decomposition of SGA into sum of blocks

In [10]:
#implements modular Fourier transform
#project v onto each block U_i = F_p[S_n]*e_i using \pi_i: v |--> v*e_i as a projection
#this is just a change of basis
def modular_fourier_transform(p,n):
    #instantiate group algebra
    SGA_GFp_n = SymmetricGroupAlgebra(GF(p),n)
    #compute the primitive central orthogonal idempotents
    idempotents = SGA_GFp_n.central_orthogonal_idempotents()
    #create a spanning set for the block corresponding to an idempotent
    spanning_set = lambda idem: [SGA_GFp_n(sigma)*idem for sigma in SGA_GFp_n.group()]
    #compute the blocks as submodules of the given spanning set
    blocks = [SGA_GFp_n.submodule(spanning_set(idem)) for idem in idempotents]
    #compute the list of basis vectors lifed to the SGA from each block
    block_decomposition_basis = flatten([[u.lift() for u in block.basis()] for block in blocks])
    #the elements of the symmetric group are ordered, giving the map from the standard basis
    sym_group_list = list(SGA_GFp_n.group())
    change_of_basis_matrix = []
    for b in block_decomposition_basis:
        coord_vector = [0]*len(sym_group_list)
        for pair in list(b):
            coord_vector[sym_group_list.index(pair[0])] = pair[1]
        change_of_basis_matrix.append(coord_vector)
    return matrix(GF(p),change_of_basis_matrix).transpose()

In [11]:
#choose characteristic p and number of elements permuted n
p=3; n=5

In [12]:
#compute the modular Fourier transform for an example element v
mft = modular_fourier_transform(p,n); mft

120 x 120 dense matrix over Finite Field of size 3 (use the '.str()' method to see the entries)

In [13]:
print(mft[0])

(1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)


In [14]:
#perform Hill cipher with MFT(p,n) as matrix
txt = "Hom encrypt key p"
bit_str = flatten([list(bin_str)[2:] for bin_str in [bin(num) for num in list(txt.encode())]])
encoded_vec = [GF(p)(bit) for bit in bit_str] + (factorial(n)-len(bit_str))*[0]
''.join([str(i) for i in mft*vector(encoded_vec)])

'111212121101000010000021200201221010010002100112011212000201222212020120111011111020020100212220212020202012102101011121'

In [16]:
modular_fourier_transform(7,3) == SGA_F7.dft()

False

In [183]:
M = modular_fourier_transform(7,4)

In [185]:
#note: this matrix has (almost) only has 1's and -1's
print(M.str())

[1 1 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 1]
[1 0 1 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 6]
[1 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 6]
[1 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 1]
[1 0 0 0 0 1 0 0 0 0 6 0 0 6 0 0 0 0 1 0 0 0 0 1]
[1 1 6 6 1 1 0 0 0 0 0 6 6 0 6 6 6 6 6 0 0 0 0 6]
[1 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 6]
[1 6 1 0 0 0 1 0 0 0 1 0 0 0 6 6 0 0 0 6 0 0 0 1]
[1 0 0 0 0 0 0 1 0 0 6 0 0 6 0 0 0 0 0 0 1 0 0 1]
[1 0 0 0 0 0 0 0 1 0 0 6 6 0 0 0 0 0 0 0 0 1 0 6]
[1 0 0 6 0 1 0 1 0 0 0 0 1 0 0 0 6 0 6 0 6 0 0 6]
[1 1 6 6 0 1 0 0 1 0 0 0 0 1 1 1 1 0 1 0 0 6 0 1]
[1 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 1 1]
[1 0 0 6 1 0 0 0 0 1 0 0 1 0 0 0 6 6 0 0 0 0 6 6]
[1 1 0 6 0 0 6 1 0 1 0 6 6 0 6 0 6 0 0 6 6 0 6 6]
[1 1 0 6 0 0 6 0 1 1 6 0 0 6 1 0 1 0 0 1 0 6 1 1]
[1 1 6 5 1 1 6 1 0 1 1 0 0 0 1 1 2 1 1 1 1 0 1 1]
[1 1 6 6 0 1 6 0 1 1 0 1 0 0 6 6 6 0 6 6 0 1 6 6]
[1 5 1 2 6 6 1 6 6 6 0 6 6 0 2 1 2 1 1 1 1 6 1 6]
[1 6 0 1 0 6 1 6 6 6 6 0 0 6 6 0 6 0 6 6 6 1 6 1]


In [99]:
matrix.identity(6) == modular_fourier_transform(2,3)^4

True

In [174]:
#A fast way for computing the order of 𝑀 is thus 
#to compute the characteristic polynomial P_L of M
#factor it over F_p and check then if prime-divisors of p^k-1
#(for 𝑘 the degree of an involved irreducible polynomial) 
#divide the order
def order_finite_field(M):
    L = M.nrows()
    p = len(M.base_ring())
    P_L = M.charpoly(); P_L
    char_poly_factored = P_L.factor(); char_poly_factored
    degree_list = [item[0].degree() for item in char_poly_factored]
    U = p^(L-1)*prod(p^k-1 for k in degree_list)
    for div in divisors(U):
        if M^(div) == matrix.identity(24):
            return div

In [175]:
order_finite_field(M)

488488

In [178]:
print((M^488488).str())

[1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]


In [22]:
SymmetricGroupAlgebra(GF(7),3).dft()

[1 1 1 1 1 1]
[1 4 6 3 3 4]
[0 6 0 6 1 1]
[0 1 0 6 1 6]
[1 3 1 3 3 3]
[1 6 6 1 1 6]

In [29]:
P = Primes()

In [30]:
P.unrank(4)

11

In [31]:
#note: this matrix has (almost) only has 1's and -1's
print(modular_fourier_transform(11,4).str())

[ 1  1  0  0  0  0  0  0  0  0  1  0  0  0  1  0  0  0  0  0  0  0  0  1]
[ 1  0  1  0  0  0  0  0  0  0  0  1  0  0  0  1  0  0  0  0  0  0  0 10]
[ 1  0  0  1  0  0  0  0  0  0  0  0  1  0  0  0  1  0  0  0  0  0  0 10]
[ 1  0  0  0  1  0  0  0  0  0  0  0  0  1  0  0  0  1  0  0  0  0  0  1]
[ 1  0  0  0  0  1  0  0  0  0 10  0  0 10  0  0  0  0  1  0  0  0  0  1]
[ 1  1 10 10  1  1  0  0  0  0  0 10 10  0 10 10 10 10 10  0  0  0  0 10]
[ 1  0  0  0  0  0  1  0  0  0  0  1  0  0  0  0  0  0  0  1  0  0  0 10]
[ 1 10  1  0  0  0  1  0  0  0  1  0  0  0 10 10  0  0  0 10  0  0  0  1]
[ 1  0  0  0  0  0  0  1  0  0 10  0  0 10  0  0  0  0  0  0  1  0  0  1]
[ 1  0  0  0  0  0  0  0  1  0  0 10 10  0  0  0  0  0  0  0  0  1  0 10]
[ 1  0  0 10  0  1  0  1  0  0  0  0  1  0  0  0 10  0 10  0 10  0  0 10]
[ 1  1 10 10  0  1  0  0  1  0  0  0  0  1  1  1  1  0  1  0  0 10  0  1]
[ 1  0  0  0  0  0  0  0  0  1  0  0  0  1  0  0  0  0  0  0  0  0  1  1]
[ 1  0  0 10  1  0  0  0  0  1  0  0  